In [85]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json

In [86]:
df = pd.read_csv("data/charging_sessions.csv")
r, c = df.shape
print(f'The dataset has {r} rows and {c} columns')

The dataset has 66450 rows and 13 columns


In [87]:
# Set Datatypes
df['connectionTime'] = pd.to_datetime(df['connectionTime'])
df['disconnectTime'] = pd.to_datetime(df['disconnectTime'])
df['doneChargingTime'] = pd.to_datetime(df['doneChargingTime'])

df['sessionID'] = df['sessionID'].astype(str)
df['siteID'] = df['siteID'].astype(str)
df['spaceID'] = df['spaceID'].astype(str)
df['stationID'] = df['stationID'].astype(str)
#df['userID'] = df['userID'].astype(str) # For site characteristics: better use if datatype is float64

In [88]:
start_date = '2018-04-25'
end_date = '2021-01-01'

df = df[(df['connectionTime'] >= start_date) & (df['disconnectTime'] <= end_date)]
df['weekday'] = df['connectionTime'].dt.day_name()

In [89]:
# Session ID is a composite value of stationID and connectionTime, and therefor redundant
# Timezone contains only one value, and can therefor be considered meta-data
# First column cannot be used as the index because of several mistakes (e.g. "index" 1 would occur 9 times)
# First column makes no sense otherwise
df = df.drop(['sessionID', 'timezone', df.columns[0]], axis=1)

In [90]:
# Add column to represent if the vehicle was fully charged during the session or only a partial charge
# If doneChargingTime is set then the vehicle is fully charged
df['fullyCharged'] = np.where(df['doneChargingTime'].isnull(), False, True)

display(df.head())

,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,siteID,spaceID,stationID,userID,userInputs,weekday,fullyCharged
0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1,AG-3F30,1-1-179-810,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil...",Thursday,True
1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1,AG-1F01,1-1-193-825,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'mil...",Thursday,True
2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1,AG-1F03,1-1-193-829,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile...",Thursday,True
3,5e23b149f9af8b5fe4b973d2,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1,AG-1F04,1-1-193-820,1117.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile...",Thursday,True
4,5e23b149f9af8b5fe4b973d3,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1,AG-1F06,1-1-193-819,334.0,"[{'WhPerMile': 400, 'kWhRequested': 16.0, 'mil...",Thursday,True


In [91]:
# Show data types of the columns
df.dtypes

id                               object
connectionTime      datetime64[ns, UTC]
disconnectTime      datetime64[ns, UTC]
doneChargingTime    datetime64[ns, UTC]
kWhDelivered                    float64
siteID                           object
spaceID                          object
stationID                        object
userID                          float64
userInputs                       object
weekday                          object
fullyCharged                       bool
dtype: object

In [92]:
# No charge time is zero when doneChargingTime is not set, else it's the difference between disconnectTime and doneChargingTime
# For some reason we can't use pd.Timedeltas here, so we hack 0 by subtracting the connectionTime from itself
df['NoChargingTime'] = np.where(df['doneChargingTime'].isnull(), df['connectionTime'] - df['connectionTime'], df['doneChargingTime'] - df['connectionTime'])
df['NoChargingTimeMinutes'] = (df['NoChargingTime']).dt.total_seconds() / 60.0  # Convert seconds to minutes
df['NoChargingTimeHours'] = ((df['NoChargingTime']).dt.total_seconds() / 60.0) / 60.0  # Convert minutes to hours

# Charging time delta between doneChargingTime and connectionTime, if doneChargingTime is set, otherwise it's the difference between disconnectTime and connectionTime
df['ChargingTime'] = np.where(df['doneChargingTime'].isnull(), df['disconnectTime'] - df['connectionTime'], df['doneChargingTime'] - df['connectionTime'])
df['ChargingTimeMinutes'] = (df['ChargingTime']).dt.total_seconds() / 60.0  # Convert seconds to minutes
df['ChargingTimeHours'] = ((df['ChargingTime']).dt.total_seconds() / 60.0) / 60.0  # Convert minutes to hours

df['kWhPerMinute'] = df['kWhDelivered'] / df['ChargingTimeMinutes']

# Create a new column 'month' to store the month information
df['month'] = df['connectionTime'].dt.month

# Create a new column 'userRegistered' to state if user is registered or not (needed for site characteristics section)
df['userRegistered'] = np.where(df['userID'].isnull(), False, True)

In [93]:
# Show null values per column
df.isnull().sum()

id                           0
connectionTime               0
disconnectTime               0
doneChargingTime          2630
kWhDelivered                 0
siteID                       0
spaceID                      0
stationID                    0
userID                   16811
userInputs               16811
weekday                      0
fullyCharged                 0
NoChargingTime               0
NoChargingTimeMinutes        0
NoChargingTimeHours          0
ChargingTime                 0
ChargingTimeMinutes          0
ChargingTimeHours            0
kWhPerMinute                 0
month                        0
userRegistered               0
dtype: int64

### Flatten the Json-Object UserInput

In [94]:
# df_charging['userInputs'] = df_charging['userInputs'].str.replace("'", '"')
# df_charging['userInputs'] = df_charging['userInputs'].str.replace('"[', "'[")
# df_charging['userInputs'] = df_charging['userInputs'].str.replace(']"', "]'")
# df_charging['userInputs'] = df_charging['userInputs'].str.replace("True", "true")
# df_charging['userInputs'] = df_charging['userInputs'].str.replace("False", "false")

# Mapping: Value to be replaced: replacing value
#replacements = {
#    "'": '"', 
#    '"[': "'[",
#    ']"': "]'",
#    "True": "true",
#    "False": "false"
# }

# shorthand to replace multiple values
# for i, j in replacements.items():
#    df_charging['userInputs'] = df_charging['userInputs'].str.replace(i, j)
# Old Version
# for i in range(len(df_charging['userInputs'])):
#     if not pd.isna(df_charging['userInputs'].iloc[i]):
#        df_charging.at[i, 'userInputs'] = json.loads(df_charging['userInputs'].iloc[i])

# Refactored Version
# df_charging['userInputs'] = df_charging['userInputs'].apply(lambda x: json.loads(x) if pd.notna(x) else x)

In [95]:
df['userInputs'] = df['userInputs'].str.replace("'", '"')
df['userInputs'] = df['userInputs'].str.replace('"[', "'[")
df['userInputs'] = df['userInputs'].str.replace(']"', "]'")
df['userInputs'] = df['userInputs'].str.replace("True", "true")
df['userInputs'] = df['userInputs'].str.replace("False", "false")

# Refactored Version
df['userInputs'] = df['userInputs'].apply(lambda x: json.loads(x) if pd.notna(x) else x)
# Extract 'paymentRequired' values and save them in a new column
df['user_paymentRequired_values'] = df['userInputs'].apply(lambda x: x[0]['paymentRequired'] if isinstance(x, list) and len(x) > 0 else None)
df['user_userID'] = df['userInputs'].apply(lambda x: x[0]['userID'] if isinstance(x, list) and len(x) > 0 else None)
df['user_requestedDeparture'] = df['userInputs'].apply(lambda x: x[0]['requestedDeparture'] if isinstance(x, list) and len(x) > 0 else None)
df['user_modifiedAt'] = df['userInputs'].apply(lambda x: x[0]['modifiedAt'] if isinstance(x, list) and len(x) > 0 else None)
df['user_minutesAvailable'] = df['userInputs'].apply(lambda x: x[0]['minutesAvailable'] if isinstance(x, list) and len(x) > 0 else None)
df['user_milesRequested'] = df['userInputs'].apply(lambda x: x[0]['milesRequested'] if isinstance(x, list) and len(x) > 0 else None)
df['user_kWhRequested'] = df['userInputs'].apply(lambda x: x[0]['kWhRequested'] if isinstance(x, list) and len(x) > 0 else None)
df['user_WhPerMile'] = df['userInputs'].apply(lambda x: x[0]['WhPerMile'] if isinstance(x, list) and len(x) > 0 else None)

# Drop the 'userInputs' column
df.drop(columns=['userInputs'], inplace=True)

# Drop the "user_userID" column, as it has the same values as the column "userID", therefore redundant
df = df.drop(['user_userID'], axis=1)

In [96]:
# Add a column 'user_kWhRequestFulfilment', which shows the difference between kWh request and delivery
df['user_kWhRequestFulfilment'] = df['kWhDelivered'] - df['user_kWhRequested']

In [97]:
# Problem detected in KPI section: connectionTime sometimes later than doneChargingTime -> negative values not possible
# Drop rows, where charging time (in hours) has negative values
df = df.drop(df[df['ChargingTimeHours'] < 0].index)

# Problem detected in KPI section: doneChargingTime sometimes later than disconnectTime -> negative values not possible
# Drop rows, where no charging time (in hours) has negative values
df = df.drop(df[df['NoChargingTimeHours'] < 0].index)

In [98]:
# Sorts the values by connection time and updates the index
df = df.sort_values('connectionTime')
df = df.reset_index(drop=True)

In [99]:
start_date = df['connectionTime'].min()
end_date = df['disconnectTime'].max()
datetime_range = pd.date_range(start=start_date, end=end_date, freq='H')

missing_periods = []
current_period = []

for datetime in datetime_range:
    no_entries_for_datetime = df[(df['connectionTime'] <= datetime) & (df['disconnectTime'] >= datetime)].empty

    if no_entries_for_datetime:
        if not current_period or datetime == current_period[-1] + pd.Timedelta(hours=1):
            current_period.append(datetime)
        else:
            if len(current_period) > 7*24:
                missing_periods.append(current_period)
            current_period = [datetime]
    else:
        if current_period and len(current_period) > 7*24:
            missing_periods.append(current_period)
        current_period = []

if current_period and len(current_period) > 7*24:
    missing_periods.append(current_period)

for period in missing_periods:
    print(f"Missing entries from {period[0]} to {period[-1]} (Duration: {len(period)/24} days)")

Missing entries from 2020-08-04 05:08:04+00:00 to 2020-11-18 20:08:04+00:00 (Duration: 106.66666666666667 days)


In [100]:
# Display first 5 rows of the dataset
display(df.head())

# Display 5 rows of dataset where the user is registered
display(df[df['userRegistered'] == True].head())

# Display data types of the columns
display(df.dtypes)

,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,siteID,spaceID,stationID,userID,weekday,...,month,userRegistered,user_paymentRequired_values,user_requestedDeparture,user_modifiedAt,user_minutesAvailable,user_milesRequested,user_kWhRequested,user_WhPerMile,user_kWhRequestFulfilment
0,5bc90cb9f9af8b0d7fe77cd2,2018-04-25 11:08:04+00:00,2018-04-25 13:20:10+00:00,2018-04-25 13:21:10+00:00,7.932,2,CA-496,2-39-78-362,NaN,Wednesday,...,4,False,None,None,None,NaN,NaN,NaN,NaN,NaN
1,5bc90cb9f9af8b0d7fe77cd3,2018-04-25 13:45:10+00:00,2018-04-26 00:56:16+00:00,2018-04-25 16:44:15+00:00,10.013,2,CA-319,2-39-95-27,NaN,Wednesday,...,4,False,None,None,None,NaN,NaN,NaN,NaN,NaN
2,5bc90cb9f9af8b0d7fe77cd4,2018-04-25 13:45:50+00:00,2018-04-25 23:04:45+00:00,2018-04-25 14:51:44+00:00,5.257,2,CA-489,2-39-79-380,NaN,Wednesday,...,4,False,None,None,None,NaN,NaN,NaN,NaN,NaN
3,5bc90cb9f9af8b0d7fe77cd5,2018-04-25 14:37:06+00:00,2018-04-25 23:55:34+00:00,2018-04-25 16:05:22+00:00,5.177,2,CA-327,2-39-79-379,NaN,Wednesday,...,4,False,None,None,None,NaN,NaN,NaN,NaN,NaN
4,5bc90cb9f9af8b0d7fe77cd6,2018-04-25 14:40:34+00:00,2018-04-25 23:03:12+00:00,2018-04-25 17:40:30+00:00,10.119,2,CA-490,2-39-79-381,NaN,Wednesday,...,4,False,None,None,None,NaN,NaN,NaN,NaN,NaN


,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,siteID,spaceID,stationID,userID,weekday,...,month,userRegistered,user_paymentRequired_values,user_requestedDeparture,user_modifiedAt,user_minutesAvailable,user_milesRequested,user_kWhRequested,user_WhPerMile,user_kWhRequestFulfilment
222,5bc915caf9af8b0dad3c0660,2018-04-30 15:07:49+00:00,2018-05-01 00:27:55+00:00,2018-05-01 00:27:51+00:00,47.808,2,CA-314,2-39-88-24,22.0,Monday,...,4,True,True,"Tue, 01 May 2018 00:17:49 GMT","Mon, 30 Apr 2018 15:08:54 GMT",550.0,170.0,59.5,350.0,-11.692
581,5bc917d0f9af8b0dc677b8bb,2018-05-07 14:38:18+00:00,2018-05-08 01:15:01+00:00,2018-05-08 01:14:53+00:00,27.683,2,CA-303,2-39-139-28,61.0,Monday,...,5,True,True,"Mon, 07 May 2018 15:38:18 GMT","Mon, 07 May 2018 14:38:32 GMT",60.0,20.0,8.0,400.0,19.683
842,5bc9190ff9af8b0dc677b9c1,2018-05-11 15:17:01+00:00,2018-05-12 00:18:43+00:00,2018-05-11 23:05:56+00:00,17.485,2,CA-314,2-39-88-24,22.0,Friday,...,5,True,True,"Sat, 12 May 2018 00:23:01 GMT","Fri, 11 May 2018 15:18:23 GMT",546.0,50.0,17.5,350.0,-0.015
955,5bc919c3f9af8b0dc677ba32,2018-05-14 13:50:26+00:00,2018-05-14 23:09:45+00:00,2018-05-14 20:43:40+00:00,11.795,2,CA-489,2-39-79-380,66.0,Monday,...,5,True,True,"Mon, 14 May 2018 14:50:26 GMT","Mon, 14 May 2018 13:51:52 GMT",60.0,20.0,8.0,400.0,3.795
1003,5bc919c3f9af8b0dc677ba62,2018-05-15 00:06:39+00:00,2018-05-15 01:13:58+00:00,2018-05-15 00:39:01+00:00,3.076,2,CA-303,2-39-139-28,61.0,Tuesday,...,5,True,True,"Tue, 15 May 2018 10:17:39 GMT","Tue, 15 May 2018 00:06:56 GMT",611.0,20.0,5.0,250.0,-1.924


id                                          object
connectionTime                 datetime64[ns, UTC]
disconnectTime                 datetime64[ns, UTC]
doneChargingTime               datetime64[ns, UTC]
kWhDelivered                               float64
siteID                                      object
spaceID                                     object
stationID                                   object
userID                                     float64
weekday                                     object
fullyCharged                                  bool
NoChargingTime                     timedelta64[ns]
NoChargingTimeMinutes                      float64
NoChargingTimeHours                        float64
ChargingTime                       timedelta64[ns]
ChargingTimeMinutes                        float64
ChargingTimeHours                          float64
kWhPerMinute                               float64
month                                        int32
userRegistered                 

### Clean-up datatypes

In [101]:
df = df.astype(
    {
        'siteID': 'int32',
        'spaceID': 'category',
        'stationID': 'category',
        'user_paymentRequired_values': 'bool',
        'user_minutesAvailable': 'float64',
        'user_milesRequested': 'float64',
        'user_kWhRequested': 'float64',
        'user_WhPerMile': 'float64'
    }

)

# Display data types of the columns
display(df.dtypes)

id                                          object
connectionTime                 datetime64[ns, UTC]
disconnectTime                 datetime64[ns, UTC]
doneChargingTime               datetime64[ns, UTC]
kWhDelivered                               float64
siteID                                       int32
spaceID                                   category
stationID                                 category
userID                                     float64
weekday                                     object
fullyCharged                                  bool
NoChargingTime                     timedelta64[ns]
NoChargingTimeMinutes                      float64
NoChargingTimeHours                        float64
ChargingTime                       timedelta64[ns]
ChargingTimeMinutes                        float64
ChargingTimeHours                          float64
kWhPerMinute                               float64
month                                        int32
userRegistered                 

In [102]:
# Drop Columns that are not needed for the analysis
df = df.drop(['id', 'user_modifiedAt'], axis=1)

display(df.head())

,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,siteID,spaceID,stationID,userID,weekday,fullyCharged,...,kWhPerMinute,month,userRegistered,user_paymentRequired_values,user_requestedDeparture,user_minutesAvailable,user_milesRequested,user_kWhRequested,user_WhPerMile,user_kWhRequestFulfilment
0,2018-04-25 11:08:04+00:00,2018-04-25 13:20:10+00:00,2018-04-25 13:21:10+00:00,7.932,2,CA-496,2-39-78-362,NaN,Wednesday,True,...,0.059594,4,False,False,None,NaN,NaN,NaN,NaN,NaN
1,2018-04-25 13:45:10+00:00,2018-04-26 00:56:16+00:00,2018-04-25 16:44:15+00:00,10.013,2,CA-319,2-39-95-27,NaN,Wednesday,True,...,0.055913,4,False,False,None,NaN,NaN,NaN,NaN,NaN
2,2018-04-25 13:45:50+00:00,2018-04-25 23:04:45+00:00,2018-04-25 14:51:44+00:00,5.257,2,CA-489,2-39-79-380,NaN,Wednesday,True,...,0.079772,4,False,False,None,NaN,NaN,NaN,NaN,NaN
3,2018-04-25 14:37:06+00:00,2018-04-25 23:55:34+00:00,2018-04-25 16:05:22+00:00,5.177,2,CA-327,2-39-79-379,NaN,Wednesday,True,...,0.058652,4,False,False,None,NaN,NaN,NaN,NaN,NaN
4,2018-04-25 14:40:34+00:00,2018-04-25 23:03:12+00:00,2018-04-25 17:40:30+00:00,10.119,2,CA-490,2-39-79-381,NaN,Wednesday,True,...,0.056237,4,False,False,None,NaN,NaN,NaN,NaN,NaN


### Save the modified dataset

In [103]:
# Save to pickle as we have already fixed the datatypes
df.to_pickle('data/charging_modified.pkl')
df.to_csv('data/charging_session_flattened_json.csv')